# Домашнее задание

Задание 1. Обучите нейронную сеть решать шифр Цезаря

1.Напишите алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)

2.Сделайте нейронную сеть

3.Обучите её (вход - зашифрованная фраза, выход - дешифрованная фраза)

4.Проверьте качество

Задание 2.Выполните практическую работу из лекционного ноутбука.

1.Постройте RNN-ячейку на основе полносвязных слоёв

2.Примените построенную ячейку для генерации текста с выражениями героев сериала "Симпсоны"

In [1]:
import pandas as pd
import time
import torch

In [13]:
word = 'привет'

In [11]:
def encod_caesar(word, move=2):
    alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    word_list = list(word.lower())
    new_word_list = []
    
    for char in word_list:
        ind = alphabet.index(char) + move

        if ind >= len(alphabet):
            new_char = alphabet[ind - len(alphabet)]
        else:
            new_char = alphabet[ind]
        
        new_word_list.append(new_char)
    
    return ''.join(new_word_list)

In [12]:
def decod_caesar(word, move=2):
    alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    word_list = list(word.lower())
    new_word_list = []
    
    for char in word_list:
        ind = alphabet.index(char) - move

        if ind < 0:
            new_char = alphabet[ind + len(alphabet)]
        
        else:
            new_char = alphabet[ind]
        
        new_word_list.append(new_char)
    
    return ''.join(new_word_list)

In [14]:
encode_word = encod_caesar(word)
encode_word

'сткджф'

In [17]:
print(decod_caesar(encode_word))

привет


In [21]:
import re

In [24]:
with open('onegin.txt', encoding='utf-8', newline='') as file:
    text = file.readlines()

In [25]:
clear_text = []
    
for item in text:
    sub = re.sub(r'[^а-я]','', item.lower())
    if len(sub) > 20:
        clear_text.append(list(sub))

In [26]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [27]:
MAX_LEN = 50
X = torch.zeros((len(clear_text), MAX_LEN), dtype=int)
for i in range(len(clear_text)):
    for j, w in enumerate(clear_text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [28]:
#кодируем

clear_encode_text = []
    
for item in text:
    sub = re.sub(r'[^а-я]','', item.lower())
    if len(sub) > 20:
        sub = encod_caesar(sub)
        clear_encode_text.append(list(sub))

In [29]:
Y = torch.zeros((len(clear_encode_text), MAX_LEN), dtype=int)
for i in range(len(clear_encode_text)):
    for j, w in enumerate(clear_encode_text[i]):
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [30]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear= torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, word, state=None):
        embed = self.embed(word)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [31]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [32]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for x, y in zip(X, Y):
        optimizer.zero_grad()
        answers = model.forward(x)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 4.088, Train loss: 0.403
Epoch 1. Time: 3.841, Train loss: 0.042
Epoch 2. Time: 3.901, Train loss: 0.021
Epoch 3. Time: 3.848, Train loss: 0.014
Epoch 4. Time: 3.721, Train loss: 0.010
Epoch 5. Time: 3.884, Train loss: 0.008
Epoch 6. Time: 3.748, Train loss: 0.006
Epoch 7. Time: 3.722, Train loss: 0.005
Epoch 8. Time: 3.870, Train loss: 0.004
Epoch 9. Time: 3.734, Train loss: 0.004


# Задание 2

In [34]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [35]:
words = df['normalized_text'].tolist()

In [36]:
text = [[c for c in word] for word in words if type(word) is str]

In [37]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [38]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [79]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        #self.drop = torch.nn.Dropout(0,5)
        self.rnn = torch.nn.RNN(28, 128)
        self.drop = torch.nn.Dropout(0,5)
        self.out= torch.nn.Linear(128, 28)
        
    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        self.relu = torch.nn.ReLU()
        x, s = self.rnn(x)
        return self.out(x)


In [80]:
model = Network()

In [81]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [82]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    
    x = torch.zeros((1, len(sentence)), dtype=int)
    
    for j, w in enumerate(sentence):
        if j >= MAX_LEN:
            break
        else:
            x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0)], axis=1)
        ww = INDEX_TO_CHAR[w]
        
        if ww == 'none':
            break
            
        sentence.append(ww)
    
    print(' '.join(sentence))

In [83]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()

Epoch 0. Time: 2.271, Train loss: 2.173
h e l l o u   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t  
Epoch 1. Time: 2.257, Train loss: 1.873
h e l l o u   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e  
Epoch 2. Time: 2.254, Train loss: 1.813
h e l l o u   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e  
Epoch 3. Time: 2.287, Train loss: 1.781
h e l l o u   w a n
Epoch 4. Time: 2.287, Train loss: 1.760
h e l l o u   w a n
Epoch 5. Time: 2.308, Train loss: 1.744
h e l l o u   w a n g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g
Epoch 6. Time: 2.279, Train loss: 1.731
h e l l o u   w a n g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g g
Epoch 7. Time: 2.290, Train loss: 1.721
h e l l o u   w h e   w h e   w h e   w h e   w h e   w h e   w h e   w h e   w h e   w h 